In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

import pandas as pd
import re
import time
from tqdm.notebook import tqdm
import time

import warnings
warnings.filterwarnings('ignore')

### Парсим список каналов

Берем произвольный сайт-каталог telegram каналов, из него достаем основную информацию о канале

In [2]:
url = 'https://in.tgstat.com/ru/tag/investments?ysclid=ltxecd5p1e666733084'
res = requests.get(url, headers={'User-Agent': UserAgent().chrome})

In [3]:
soup = BeautifulSoup(res.content)

In [4]:
channel_blocks = soup.find_all('div', {'class':r'col-12 col-sm-6 col-lg-4'})

In [5]:
names = []
subscribers = []
links = []
disc = []

for block in channel_blocks:
    names.append(block.find('div', {'class' : r'font-16 text-dark text-truncate'}).text)
    subscribers.append(int(block.find('div', {'class' : r'font-12 text-truncate'})\
                      .text.strip()[:-12].replace(' ', '')))
    disc.append(block.find('div', attrs={'class' : r'font-14 text-muted line-clamp-2 mt-1'}).text.strip())
    link = block.find(attrs={'class' : r'text-body'}, href=True)['href']
    
    dog_sigh_pos = link.find('@')
    slash_pos = [m.start() for m in re.finditer('/', link)][-1]
    
    if dog_sigh_pos != -1:
        link = 'https://t.me/' + link[dog_sigh_pos+1:]
    else:
        link = 'https://t.me/' + link[slash_pos+1:]
        
    links.append(link)

In [6]:
channels_info = pd.DataFrame({'name' : names,
                             'subscribers' : subscribers, 
                             'link' : links, 
                             'description' : disc})

print(channels_info.shape)
channels_info.head()

(102, 4)


,name,subscribers,link,description
0,Тинькофф Инвестиции,670825,https://t.me/tinkoff_invest_official,Официальный канал Тинькофф.
1,Альфа-Инвестиции,415942,https://t.me/alfa_investments,Официальный канал Альфа-Инвестиций с инвестиде...
2,Мои Инвестиции,286117,https://t.me/omyinvestments,"Инвестидеи, обзоры рынка и многое другое от лу..."
3,bitkogan,284405,https://t.me/bitkogan,Авторский канал Евгения Когана
4,СберИнвестиции,232751,https://t.me/SberInvestments,"Канал СберИнвестиции, в котором мы рассказывае..."


In [7]:
from telethon import TelegramClient
import config

In [8]:
api_id = config.API_ID
api_hash = config.API_HASH
name = 'test'

In [9]:
client = TelegramClient(name, api_id, api_hash)
await client.start()

In [10]:
from telethon.tl.functions.messages import GetHistoryRequest

In [11]:
cnt = 0
msg = []
channels_info['is_restricted'] = True
channels_info['error_while_parsing'] = False
channels_info['create_date'] = None


for i, channel in enumerate(tqdm(channels_info.to_dict('records'))):
    try:
        url = channel['link']
        tg_channel = await client.get_entity(url)
        channels_info.at[i, 'create_date'] = pd.to_datetime(tg_channel.to_dict()['date']).tz_localize(None)

        history = await client(GetHistoryRequest(
            peer=tg_channel,
            offset_id=0,
            offset_date=None, add_offset=0,
            limit=1, max_id=0, min_id=0,
            hash=0))

    except:
        channels_info.at[i, 'error_while_parsing'] = True
        
    time.sleep(1)

  0%|          | 0/102 [00:00<?, ?it/s]

In [12]:
channels_info = channels_info[~channels_info['error_while_parsing']]\
                            [['name', 'subscribers', 'link', 'description', 'create_date']]\
                            .reset_index().rename({'index' : 'channel_id'}, axis=1)
print(len(channels_info))
channels_info.head(10)

71


,channel_id,name,subscribers,link,description,create_date
0,0,Тинькофф Инвестиции,670825,https://t.me/tinkoff_invest_official,Официальный канал Тинькофф.,2019-09-04 15:50:32
1,1,Альфа-Инвестиции,415942,https://t.me/alfa_investments,Официальный канал Альфа-Инвестиций с инвестиде...,2021-06-18 14:04:01
2,2,Мои Инвестиции,286117,https://t.me/omyinvestments,"Инвестидеи, обзоры рынка и многое другое от лу...",2019-09-12 12:53:25
3,3,bitkogan,284405,https://t.me/bitkogan,Авторский канал Евгения Когана,2018-01-14 14:46:07
4,4,СберИнвестиции,232751,https://t.me/SberInvestments,"Канал СберИнвестиции, в котором мы рассказывае...",2020-07-24 16:15:43
5,6,InvestFuture,194786,https://t.me/investfuture,Медиа про деньги https://t.me/addlist/OhM2a9Hh...,2017-11-15 10:25:25
6,8,Лимон на чай,152532,https://t.me/lemonfortea,"Реальный опыт инвестора, со всеми граблями и у...",2017-10-06 07:15:03
7,9,РынкиДеньгиВласть | РДВ,150076,https://t.me/AK47pfl,"аналитика по российскому рынку ценных бумаг, к...",2017-04-05 08:43:54
8,10,IF Stocks,121468,https://t.me/if_stocks,Реклама: https://t.me/IF_adv,2020-09-20 21:40:46
9,11,На пенсию в 35 лет,111125,https://t.me/pensiya35,В 35 лет мои доходы от инвестиций превысили ра...,2019-09-24 10:29:11


Итак, у нас есть 71 telegram каналов с доступом к сообщениям

In [1]:
channels_info.to_csv('../../../data/telegram/channels.csv', index=False)